## Snowflake (On-Demand) Feature Group 
This tutorial notebook will show how you can define an external feature group for a table in Snowflake.

In this notebook we assume that you already have snowflake account and did the getting started with snowflake tutorial.

In [1]:
import hsfs

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
1,application_1614610677629_0002,pyspark,idle,Link,Link


SparkSession available as 'spark'.


In [2]:
connection = hsfs.connection()
# Get the feature store handle for the project's feature store
fs = connection.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

In [ ]:
snowflake_conn = fs.get_storage_connector("sfconnector")
snowflake_conn.spark_options()
sfOptions = snowflake_conn.spark_options()
sfOptions["query"] = "select * from TELCO"

In [6]:
df = spark.read.format("net.snowflake.spark.snowflake") \
  .options(**sfOptions) \
  .load()
df.show(10)

+-----------+------+--------------+-------+----------+------+-------------+----------------+----------------+---------------+-------------+-----------------+------------+------------+----------------+--------------+-----------------+--------------------+---------------+-------------+-----+
|CUSTOMER_ID|GENDER|SENIOR_CITIZEN|PARTNER|DEPENDENTS|TENURE|PHONE_SERVICE|  MULTIPLE_LINES|INTERNET_SERVICE|ONLINE_SECURITY|ONLINE_BACKUP|DEVICE_PROTECTION|TECH_SUPPORT|STREAMING_TV|STREAMING_MOVIES|      CONTRACT|PAPERLESS_BILLING|      PAYMENT_METHOD|MONTHLY_CHARGES|TOTAL_CHARGES|CHURN|
+-----------+------+--------------+-------+----------+------+-------------+----------------+----------------+---------------+-------------+-----------------+------------+------------+----------------+--------------+-----------------+--------------------+---------------+-------------+-----+
| 7590-VHVEG|Female|         false|    Yes|        No|     1|           No|No phone service|             DSL|             No|  

### External (On-Demand) Feature Group
An on-Demand Feature Group stores the metadata for features in the feature store. The actual feature data is read from the external source using a storage connector. 
The next cell shows how to create an on-demand feature group using the snowflake connector.

In [7]:
telco_on_dmd = fs.create_on_demand_feature_group(name="telco_snowflake_scala",
                                                version=2,
                                                query="select * from telco",
                                                description="On-demand feature group for telecom customer data",
                                                storage_connector=snowflake_conn,
                                                statistics_config=True)

In [8]:
telco_on_dmd.save()

In [9]:
telco_on_dmd.select(['customer_id', 'internet_service', 'phone_service', 'total_charges', 'churn']).show(5)

+-----------+----------------+-------------+-------------+-----+
|customer_id|internet_service|phone_service|total_charges|churn|
+-----------+----------------+-------------+-------------+-----+
| 7590-VHVEG|             DSL|           No|        29.85|   No|
| 5575-GNVDE|             DSL|          Yes|       1889.5|   No|
| 3668-QPYBK|             DSL|          Yes|       108.15|  Yes|
| 7795-CFOCW|             DSL|           No|      1840.75|   No|
| 9237-HQITU|     Fiber optic|          Yes|       151.65|  Yes|
+-----------+----------------+-------------+-------------+-----+
only showing top 5 rows